### IMPORT NECESSARY LIBRARIES ###

In [64]:
# Importing necessary libraries for the movie recommender system

# Numpy for numerical operations
import numpy as np
# Pandas for data manipulation and analysis
import pandas as pd
# ast for processing abstract syntax trees (used for converting strings to lists)
import ast
# pickle for serializing and deserializing Python objects
import pickle
# nltk for natural language processing tasks
import nltk
# CountVectorizer from sklearn for converting text data into a matrix of token counts
from sklearn.feature_extraction.text import CountVectorizer
# PorterStemmer from nltk for stemming words (reducing words to their root form)
from nltk.stem.porter import PorterStemmer
# cosine_similarity from sklearn for calculating cosine similarity between vectors
from sklearn.metrics.pairwise import cosine_similarity

### LOAD DATA FILES ###

In [65]:
# Reading the movies dataset from a CSV file into a pandas DataFrame
movies = pd.read_csv('tmdb_5000_movies.csv')

# Reading the credits dataset from a CSV file into a pandas DataFrame
credits = pd.read_csv('tmdb_5000_credits.csv')

### MOVIES DATASET ###

In [66]:
# Displaying the first few rows of the 'movies' DataFrame to inspect the data
movies.head()

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950   
4  John Carter is a war-weary, former military ca...   43.926995   

                                production_companies  \
0  [{"name": "Ingenious Film Partners", "id": 289...   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...   
3  [{"name": "Legendary Pictures", "id": 923}, {"...   
4        [{"name": "Walt Disney Pictures", "id": 2}]   

                                production_countries release_date     revenue  \
0  [{"iso_3166_1": "US", "name": "United States o...   2009-12-10  2787965087   
1  [{"iso_3166_1": "US", "name": "United States o...   2007-05-19   961000000   
2  [{"iso_3166_1": "GB", "name": "United Kingdom"...   2015-10-26   880674609   
3  [{"iso_3166_1": "US", "name": "United States o...   2012-07-16  1084939099   
4  [{"iso_3166_1": "US", "name": "United States o...   2012-03-07   284139100   

   runtime                                   spoken_languages    status  \
0    162.0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1    169.0           [{"iso_639_1": "en", "name": "English"}]  Released   
2    148.0  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3    165.0           [{"iso_639_1": "en", "name": "English"}]  Released   
4    132.0           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   
3                                 The Legend Ends   
4            Lost in our world, found in another.   

                                      title  vote_average  vote_count  
0                                    Avatar           7.2       11800  
1  Pirates of the Caribbean: At World's End           6.9        4500  
2                                   Spectre           6.3        4466  
3                     The Dark Knight Rises           7.6        9106  
4                               John Carter           6.1        2124

### CREDIT DATASET ###

In [67]:
# Displaying the first few rows of the 'credits' DataFrame to inspect the data
credits.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

### SHAPE OF THE DATASETS BEFORE JOINING ###

In [68]:
# Checking the dimensions of the 'movies' DataFrame (number of rows and columns)
movies.shape

(4803, 20)

In [69]:
# Checking the dimensions of the 'credits' DataFrame (number of rows and columns)
credits.shape

(4803, 4)

### JOINING BOTH THE DATASETS ###

In [70]:
# Merging the 'movies' DataFrame with the 'credits' DataFrame based on the 'title' column
# This combines the movie details with the associated credits information
movies = movies.merge(credits, on='title')

In [71]:
# Displaying the first few rows of the merged 'movies' DataFrame to verify the results of the merge
movies.head()

budget                                             genres  \
0  237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                       homepage      id  \
0                   http://www.avatarmovie.com/   19995   
1  http://disney.go.com/disneypictures/pirates/     285   
2   http://www.sonypictures.com/movies/spectre/  206647   
3            http://www.thedarkknightrises.com/   49026   
4          http://movies.disney.com/john-carter   49529   

                                            keywords original_language  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...                en   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...                en   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...                en   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...                en   
4  [{"id": 818, "name": "based on novel"}, {"id":...                en   

                             original_title  \
0                                    Avatar   
1  Pirates of the Caribbean: At World's End   
2                                   Spectre   
3                     The Dark Knight Rises   
4                               John Carter   

                                            overview  popularity  \
0  In the 22nd century, a paraplegic Marine is di...  150.437577   
1  Captain Barbossa, long believed to be dead, ha...  139.082615   
2  A cryptic message from Bond’s past sends him o...  107.376788   
3  Following the death of District Attorney Harve...  112.312950   
4  John Carter is a war-weary, former military ca...   43.926995   

                                production_companies  ... runtime  \
0  [{"name": "Ingenious Film Partners", "id": 289...  ...   162.0   
1  [{"name": "Walt Disney Pictures", "id": 2}, {"...  ...   169.0   
2  [{"name": "Columbia Pictures", "id": 5}, {"nam...  ...   148.0   
3  [{"name": "Legendary Pictures", "id": 923}, {"...  ...   165.0   
4        [{"name": "Walt Disney Pictures", "id": 2}]  ...   132.0   

                                    spoken_languages    status  \
0  [{"iso_639_1": "en", "name": "English"}, {"iso...  Released   
1           [{"iso_639_1": "en", "name": "English"}]  Released   
2  [{"iso_639_1": "fr", "name": "Fran\u00e7ais"},...  Released   
3           [{"iso_639_1": "en", "name": "English"}]  Released   
4           [{"iso_639_1": "en", "name": "English"}]  Released   

                                          tagline  \
0                     Enter the World of Pandora.   
1  At the end of the world, the adventure begins.   
2                           A Plan No One Escapes   
3                                 The Legend Ends   
4            Lost in our world, found in another.   

                                      title vote_average vote_count movie_id  \
0                                    Avatar          7.2      11800    19995   
1  Pirates of the Caribbean: At World's End          6.9       4500      285   
2                                   Spectre          6.3       4466   206647   
3                     The Dark Knight Rises          7.6       9106    49026   
4                               John Carter          6.1       2124    49529   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...

In [72]:
# Checking the dimensions of the merged 'movies' DataFrame to understand the changes after the merge
movies.shape

(4809, 23)

In [73]:
# Displaying summary information about the 'movies' DataFrame, including the data types and non-null counts of each column
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

### CHOOSING SPECIFIC COLUMNS FOR ANALYSIS ###

In [74]:
# Selecting specific columns from the 'movies' DataFrame to keep only relevant information for the recommender system
# The selected columns are: 'movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]

In [75]:
# Displaying the first few rows of the 'movies' DataFrame to verify the selected columns and ensure the data is correctly formatted
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                              genres  \
0  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
1  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
2  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
3  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
4  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                            keywords  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...   
4  [{"id": 818, "name": "based on novel"}, {"id":...   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

### DATA PREPROCESSING ###

In [76]:
# Checking for null values in each column of the 'movies' DataFrame to identify any missing data
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [77]:
# Dropping all rows with any missing values from the 'movies' DataFrame to ensure complete data for analysis
movies.dropna(inplace=True)

In [78]:
# Verifying that there are no more missing values in the 'movies' DataFrame after dropping rows with null values
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [79]:
# Checking for duplicate rows in the 'movies' DataFrame to identify any redundant entries
movies.duplicated().sum()

0

### CREATING TAGS ###

In [80]:
# Inspecting the 'genres' column of the first row in the 'movies' DataFrame to understand its format and content
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [81]:
# The 'genres' column currently contains genre data in a string format with JSON-like structure.
# Example of genre data before transformation: '[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'
# Example of genre data after transformation: ['Action', 'Adventure', 'Fantasy', 'SciFi']

### Creating a Function to extract genres ###

### Genre has string of list value converting into a list ###

In [82]:
def convert(obj):
    # Initialize an empty list to store genre names
    list = []
    
    # Iterate over each item in the parsed object (assumed to be a JSON-like string)
    for i in ast.literal_eval(obj):
        # Append the 'name' field of each item to the list
        list.append(i['name'])
    
    # Return the list of genre names
    return list

In [83]:
# Applying the 'convert' function to the 'genres' column of the 'movies' DataFrame
# This function transforms the JSON-like string format into a list of genre names
movies['genres'] = movies['genres'].apply(convert)

In [84]:
# Displaying the first few rows of the 'movies' DataFrame to verify the transformation of the 'genres' column
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [{"id": 1463, "name": "culture clash"}, {"id":...   
1  [{"id": 270, "name": "ocean"}, {"id": 726, "na...   
2  [{"id": 470, "name": "spy"}, {"id": 818, "name...   
3  [{"id": 849, "name": "dc comics"}, {"id": 853,...   
4  [{"id": 818, "name": "based on novel"}, {"id":...   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

### Keywords has string of list value converting into a list ###

In [85]:
# Applying the 'convert' function to the 'keywords' column of the 'movies' DataFrame
# This function transforms the JSON-like string format into a list of keywords
movies['keywords'] = movies['keywords'].apply(convert)

In [86]:
# Displaying the first few rows of the 'movies' DataFrame to verify the transformation of the 'keywords' column
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                                cast  \
0  [{"cast_id": 242, "character": "Jake Sully", "...   
1  [{"cast_id": 4, "character": "Captain Jack Spa...   
2  [{"cast_id": 1, "character": "James Bond", "cr...   
3  [{"cast_id": 2, "character": "Bruce Wayne / Ba...   
4  [{"cast_id": 5, "character": "John Carter", "c...   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

### Extracting the first 3 Actor names from the Cast Column ###

In [87]:
def convert3(obj):
    # Initialize an empty list to store up to 3 genre names or keywords
    result_list = []
    # Initialize a counter to track the number of items added to the list
    counter = 0
    
    # Iterate over each item in the parsed object (assumed to be a JSON-like string)
    for i in ast.literal_eval(obj):
        # Append the 'name' field of each item to the result_list if counter is less than 3
        if counter != 3:
            result_list.append(i['name'])
            counter += 1
        else:
            # Stop iteration after adding 3 items
            break
    
    # Return the list containing up to 3 genre names or keywords
    return result_list

In [88]:
# Applying the 'convert3' function to the 'cast' column of the 'movies' DataFrame
# This function transforms the JSON-like string format into a list containing up to 3 cast member names
movies['cast'] = movies['cast'].apply(convert3)

In [89]:
# Displaying the first few rows of the 'movies' DataFrame to verify the transformation of the 'cast' column
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast  \
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]   
1     [Johnny Depp, Orlando Bloom, Keira Knightley]   
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]   
3      [Christian Bale, Michael Caine, Gary Oldman]   
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]   

                                                crew  
0  [{"credit_id": "52fe48009251416c750aca23", "de...  
1  [{"credit_id": "52fe4232c3a36847f800b579", "de...  
2  [{"credit_id": "54805967c3a36829b5002c41", "de...  
3  [{"credit_id": "52fe4781c3a36847f81398c3", "de...  
4  [{"credit_id": "52fe479ac3a36847f813eaa3", "de...

### Extracting Director Names from the Crew Column ###

In [90]:
def fetch_Director(obj):
    # Initialize an empty list to store director names
    list = []
    
    # Iterate over each item in the parsed object (assumed to be a JSON-like string)
    for i in ast.literal_eval(obj):
        # Check if the job title is 'Director'
        if i['job'] == 'Director':
            # Append the director's name to the list and stop iteration
            list.append(i['name'])
            break
    
    # Return the list containing the name of the director
    return list

In [91]:
# Applying the 'fetch_Director' function to the 'crew' column of the 'movies' DataFrame
# This function extracts the name of the director from the JSON-like string format and stores it in the 'crew' column
movies['crew'] = movies['crew'].apply(fetch_Director)

In [92]:
# Displaying the first few rows of the 'movies' DataFrame to verify the transformation of the 'crew' column and ensure directors' names are correctly extracted
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  In the 22nd century, a paraplegic Marine is di...   
1  Captain Barbossa, long believed to be dead, ha...   
2  A cryptic message from Bond’s past sends him o...   
3  Following the death of District Attorney Harve...   
4  John Carter is a war-weary, former military ca...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast                 crew  
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]      [James Cameron]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley]     [Gore Verbinski]  
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]         [Sam Mendes]  
3      [Christian Bale, Michael Caine, Gary Oldman]  [Christopher Nolan]  
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]     [Andrew Stanton]

### Overview column has string values Converting into a list to concatenate with other lists ###

In [93]:
# Applying a lambda function to the 'overview' column of the 'movies' DataFrame
# This function splits each overview text into a list of individual words
movies['overview'] = movies['overview'].apply(lambda x: x.split())

In [94]:
# Displaying the first few rows of the 'movies' DataFrame to verify the transformation of the 'overview' column into lists of words
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                          genres  \
0  [Action, Adventure, Fantasy, Science Fiction]   
1                   [Adventure, Fantasy, Action]   
2                     [Action, Adventure, Crime]   
3               [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, Science Fiction]   

                                            keywords  \
0  [culture clash, future, space war, space colon...   
1  [ocean, drug abuse, exotic island, east india ...   
2  [spy, based on novel, secret agent, sequel, mi...   
3  [dc comics, crime fighter, terrorist, secret i...   
4  [based on novel, mars, medallion, space travel...   

                                               cast                 crew  
0  [Sam Worthington, Zoe Saldana, Sigourney Weaver]      [James Cameron]  
1     [Johnny Depp, Orlando Bloom, Keira Knightley]     [Gore Verbinski]  
2      [Daniel Craig, Christoph Waltz, Léa Seydoux]         [Sam Mendes]  
3      [Christian Bale, Michael Caine, Gary Oldman]  [Christopher Nolan]  
4    [Taylor Kitsch, Lynn Collins, Samantha Morton]     [Andrew Stanton]

### REMOVING SPACES IN THE VALUES FOR (genres,keywords,cast,crew) COLUMNS FOR BETTER RECOMMENDATIONS ###

In [95]:
# Removing spaces from each genre name in the 'genres' column
movies['genres'] = movies['genres'].apply(lambda x: [i.replace(" ", "") for i in x])

# Removing spaces from each keyword in the 'keywords' column
movies['keywords'] = movies['keywords'].apply(lambda x: [i.replace(" ", "") for i in x])

# Removing spaces from each cast member name in the 'cast' column
movies['cast'] = movies['cast'].apply(lambda x: [i.replace(" ", "") for i in x])

# Removing spaces from each crew member name in the 'crew' column
movies['crew'] = movies['crew'].apply(lambda x: [i.replace(" ", "") for i in x])

In [96]:
# Displaying the first few rows of the 'movies' DataFrame to verify that spaces have been removed from 'genres', 'keywords', 'cast', and 'crew' columns
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, ScienceFiction]   

                                            keywords  \
0  [cultureclash, future, spacewar, spacecolony, ...   
1  [ocean, drugabuse, exoticisland, eastindiatrad...   
2  [spy, basedonnovel, secretagent, sequel, mi6, ...   
3  [dccomics, crimefighter, terrorist, secretiden...   
4  [basedonnovel, mars, medallion, spacetravel, p...   

                                            cast                crew  
0  [SamWorthington, ZoeSaldana, SigourneyWeaver]      [JamesCameron]  
1     [JohnnyDepp, OrlandoBloom, KeiraKnightley]     [GoreVerbinski]  
2      [DanielCraig, ChristophWaltz, LéaSeydoux]         [SamMendes]  
3      [ChristianBale, MichaelCaine, GaryOldman]  [ChristopherNolan]  
4    [TaylorKitsch, LynnCollins, SamanthaMorton]     [AndrewStanton]

### CREATING A NEW COLUMN TO CONCATENATE COLUMNS (genres,keywords,cast,crew) ###

In [97]:
# Combining the 'overview', 'genres', 'keywords', 'cast', and 'crew' columns to create a new 'tags' column
# The 'tags' column will contain a comprehensive list of all relevant textual information for each movie
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [98]:
# Displaying the first few rows of the 'movies' DataFrame to verify the creation and content of the new 'tags' column
movies.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                            overview  \
0  [In, the, 22nd, century,, a, paraplegic, Marin...   
1  [Captain, Barbossa,, long, believed, to, be, d...   
2  [A, cryptic, message, from, Bond’s, past, send...   
3  [Following, the, death, of, District, Attorney...   
4  [John, Carter, is, a, war-weary,, former, mili...   

                                         genres  \
0  [Action, Adventure, Fantasy, ScienceFiction]   
1                  [Adventure, Fantasy, Action]   
2                    [Action, Adventure, Crime]   
3              [Action, Crime, Drama, Thriller]   
4           [Action, Adventure, ScienceFiction]   

                                            keywords  \
0  [cultureclash, future, spacewar, spacecolony, ...   
1  [ocean, drugabuse, exoticisland, eastindiatrad...   
2  [spy, basedonnovel, secretagent, sequel, mi6, ...   
3  [dccomics, crimefighter, terrorist, secretiden...   
4  [basedonnovel, mars, medallion, spacetravel, p...   

                                            cast                crew  \
0  [SamWorthington, ZoeSaldana, SigourneyWeaver]      [JamesCameron]   
1     [JohnnyDepp, OrlandoBloom, KeiraKnightley]     [GoreVerbinski]   
2      [DanielCraig, ChristophWaltz, LéaSeydoux]         [SamMendes]   
3      [ChristianBale, MichaelCaine, GaryOldman]  [ChristopherNolan]   
4    [TaylorKitsch, LynnCollins, SamanthaMorton]     [AndrewStanton]   

                                                tags  
0  [In, the, 22nd, century,, a, paraplegic, Marin...  
1  [Captain, Barbossa,, long, believed, to, be, d...  
2  [A, cryptic, message, from, Bond’s, past, send...  
3  [Following, the, death, of, District, Attorney...  
4  [John, Carter, is, a, war-weary,, former, mili...

### Creating a New DataFrame of 3 Columns (movies_id,title,tag) ###

In [99]:
# Creating a new DataFrame 'new_df' that includes only the 'movie_id', 'title', and 'tags' columns from the 'movies' DataFrame
new_df = movies[['movie_id', 'title', 'tags']]

In [100]:
# Displaying the content of the 'new_df' DataFrame to review its structure and verify the selected columns
new_df

movie_id                                     title  \
0        19995                                    Avatar   
1          285  Pirates of the Caribbean: At World's End   
2       206647                                   Spectre   
3        49026                     The Dark Knight Rises   
4        49529                               John Carter   
...        ...                                       ...   
4804      9367                               El Mariachi   
4805     72766                                 Newlyweds   
4806    231617                 Signed, Sealed, Delivered   
4807    126186                          Shanghai Calling   
4808     25975                         My Date with Drew   

                                                   tags  
0     [In, the, 22nd, century,, a, paraplegic, Marin...  
1     [Captain, Barbossa,, long, believed, to, be, d...  
2     [A, cryptic, message, from, Bond’s, past, send...  
3     [Following, the, death, of, District, Attorney...  
4     [John, Carter, is, a, war-weary,, former, mili...  
...                                                 ...  
4804  [El, Mariachi, just, wants, to, play, his, gui...  
4805  [A, newlywed, couple's, honeymoon, is, upended...  
4806  ["Signed,, Sealed,, Delivered", introduces, a,...  
4807  [When, ambitious, New, York, attorney, Sam, is...  
4808  [Ever, since, the, second, grade, when, he, fi...  

[4806 rows x 3 columns]

### CONVERTING A TAG COLUMN WHICH IS A LIST INTO A STRING ###

In [101]:
# Converting the lists in the 'tags' column into a single string for each movie by joining the elements with a space
# This transformation prepares the 'tags' column for further text processing
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [102]:
# Displaying the first few rows of the 'new_df' DataFrame to verify the transformation of the 'tags' column into a single string for each movie
new_df.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                tags  
0  In the 22nd century, a paraplegic Marine is di...  
1  Captain Barbossa, long believed to be dead, ha...  
2  A cryptic message from Bond’s past sends him o...  
3  Following the death of District Attorney Harve...  
4  John Carter is a war-weary, former military ca...

In [103]:
# Converting the 'tags' column to lowercase and removing any extra spaces between words by splitting and rejoining the text
# This standardizes the 'tags' column for text processing
new_df.loc[:, 'tags'] = new_df['tags'].apply(lambda x: " ".join(x.split()).lower())

In [104]:
# Displaying the first few rows of the 'new_df' DataFrame to verify that the 'tags' column has been converted to lowercase and whitespace has been normalized
new_df.head()

movie_id                                     title  \
0     19995                                    Avatar   
1       285  Pirates of the Caribbean: At World's End   
2    206647                                   Spectre   
3     49026                     The Dark Knight Rises   
4     49529                               John Carter   

                                                tags  
0  in the 22nd century, a paraplegic marine is di...  
1  captain barbossa, long believed to be dead, ha...  
2  a cryptic message from bond’s past sends him o...  
3  following the death of district attorney harve...  
4  john carter is a war-weary, former military ca...

### CONVERTING THE TAGS INTO VECTORS ###

### REMOVING STOPWORDS ###

In [105]:
# Initializing the CountVectorizer to convert text data into a matrix of token counts
# 'max_features=5000' limits the vocabulary to the top 5000 most frequent words
# 'stop_words='english'' removes common English stop words from the tokenization process
cv = CountVectorizer(max_features=5000, stop_words='english')

### CONVERTING EACH TAG INTO A NUMPY ARRAY ###

In [106]:
# Fitting the CountVectorizer on the 'tags' column of 'new_df' and transforming the text data into a matrix of token counts
# Converting the resulting sparse matrix into a dense array
cv.fit_transform(new_df['tags']).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [107]:
# Transforming the 'tags' column into a matrix of token counts and converting it to a dense array
# Checking the shape of the resulting array to understand its dimensions (number of rows and features)
cv.fit_transform(new_df['tags']).toarray().shape

(4806, 5000)

In [108]:
# Transforming the 'tags' column into a matrix of token counts and converting it to a dense array
# Storing the resulting array in the variable 'vectors' for further processing
vectors = cv.fit_transform(new_df['tags']).toarray()

In [109]:
# Displaying the 'vectors' variable to review the matrix of token counts obtained from the 'tags' column
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

### WHAT ARE THE 5000 MOST FREQUENT WORDS ###

In [113]:
# Retrieving the feature names (i.e., the words) generated by the CountVectorizer
# These feature names correspond to the columns in the 'vectors' matrix
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

### APPLYINHG STEMMING TO CONVERT SAME WORDS INTO A SINGLE i.e. (act,acting) = (act) ###

In [114]:
# Initializing the PorterStemmer for stemming words to their root forms
# This helps in reducing variations of words to a common base form for better text analysis
ps = PorterStemmer()

### CREATING A FUNCTION TO PROCESS EACH STRING INTO A LIST ###

In [115]:
def stem(text):
    # Initialize an empty list to store the stemmed words
    y = []
    
    # Iterate over each word in the input text
    for i in text.split():
        # Apply stemming to each word and append the result to the list
        y.append(ps.stem(i))
    
    # Join the stemmed words into a single string and return it
    return " ".join(y)

### STEMMING THE TAGS COLUMN ###

In [116]:
# Applying the 'stem' function to the 'tags' column of 'new_df'
# This function stems each word in the 'tags' column to its root form
new_df.loc[:, 'tags'] = new_df['tags'].apply(stem)

In [117]:
# Displaying the 'tags' column of the 'new_df' DataFrame to verify the results of stemming
new_df['tags']

0       in the 22nd century, a parapleg marin is dispa...
1       captain barbossa, long believ to be dead, ha c...
2       a cryptic messag from bond’ past send him on a...
3       follow the death of district attorney harvey d...
4       john carter is a war-weary, former militari ca...
                              ...                        
4804    el mariachi just want to play hi guitar and ca...
4805    a newlyw couple' honeymoon is upend by the arr...
4806    "signed, sealed, delivered" introduc a dedic q...
4807    when ambiti new york attorney sam is sent to s...
4808    ever sinc the second grade when he first saw h...
Name: tags, Length: 4806, dtype: object

### CALCULATING THE SIMILARITY FOR EACH VECTOR ( MOVIE ) TO ALL THE OTHER ###

In [118]:
# Calculating the cosine similarity matrix for the 'vectors' array
# The matrix will contain similarity scores between each pair of movies based on their 'tags'
similarity = cosine_similarity(vectors)

In [119]:
# Checking the shape of the 'similarity' matrix to understand its dimensions (number of movies and similarity scores)
similarity.shape

(4806, 4806)

### CREATING A RECOMMENDER FUNCTION TO EXTRACT TO 5 SIMILAR MOVIES ###

In [120]:
def recommend(movie):
    # Finding the index of the movie in the DataFrame based on the movie title
    movie_index = new_df[new_df['title'] == movie].index[0]
    
    # Retrieving the similarity scores for the given movie
    distances = similarity[movie_index]
    
    # Sorting the movies by similarity scores in descending order, excluding the input movie itself
    # Selecting the top 5 most similar movies
    movie_list = sorted(list(enumerate(distances)), reverse=True, key=lambda x: x[1])[1:6]
    
    # Printing the titles of the top 5 most similar movies
    for i in movie_list:
        print(new_df.iloc[i[0]].title)

In [121]:
# Calling the 'recommend' function to get and print the titles of movies similar to 'Batman'
recommend('Batman')

Batman
Batman & Robin
The Dark Knight Rises
Batman Begins
Batman Returns


### SAVING THE MOVIES MODEL ###

In [122]:
# Serializing the 'new_df' DataFrame into a dictionary format and saving it to a file named 'movies_dict.pkl'
# The file is opened in write-binary mode ('wb') for storing the serialized object
pickle.dump(new_df.to_dict(), open('movies_dict.pkl', 'wb'))

### TTILE OF EACH MOVIE IN THE DATASET ###

In [123]:
# Retrieving the array of movie titles from the 'title' column of the 'new_df' DataFrame
# This provides a list of all movie titles present in the DataFrame
new_df['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

### SAVING THE SIMILARITY MODEL ###

In [124]:
# Serializing the 'similarity' matrix and saving it to a file named 'similarity.pkl'
# The file is opened in write-binary mode ('wb') for storing the serialized object
pickle.dump(similarity, open('similarity.pkl', 'wb'))